In [1]:
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # specify the path of the folder containing "file_name" :
    path_to_file = '/content/gdrive/My Drive/BT5153_Project'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/BT5153_Project
/content/gdrive/My Drive/BT5153_Project


In [2]:
#import packages
import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [3]:
df = pd.read_csv('london_nearest_tube.csv')
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,station,station_dist,walking_distance
0,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.568610,-0.112700,Private room,65,...,22,2021-11-21,0.16,2,361,1,NaN,Crouch Hill,431.108995,2.0
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,NaN,Kensington and Chelsea,51.487800,-0.168130,Entire home/apt,75,...,89,2020-03-16,0.61,1,4,0,NaN,South Kensington,805.406363,0.0
2,17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,67564,Liz,NaN,Westminster,51.521950,-0.140940,Entire home/apt,265,...,43,2021-06-28,0.33,1,295,1,NaN,Great Portland Street,292.948350,5.0
3,17506,Boutique Chelsea/Fulham Double bed 5-star ensuite,67915,Charlotte,NaN,Hammersmith and Fulham,51.479350,-0.197430,Private room,150,...,0,NaN,NaN,2,364,0,NaN,Fulham Broadway,165.612825,2.0
4,25123,Clean big Room in London (Room 1),103583,Grace,NaN,Barnet,51.574380,-0.210810,Private room,29,...,129,2020-03-15,0.91,3,0,0,NaN,Brent Cross,302.665825,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66636,53709893,Private 3 bedroom Crystal palace in high street,96150452,Thananthorn(May),NaN,Croydon,51.419345,-0.079681,Entire home/apt,120,...,0,NaN,NaN,13,253,0,NaN,Crystal Palace,509.159633,2.0
66637,53710986,"Chiswick Park 3 bed 2 bath, Chiswick, West London",125392965,Lawrence,NaN,Ealing,51.502002,-0.276895,Entire home/apt,288,...,0,NaN,NaN,10,260,0,NaN,Acton Town,264.806903,2.0
66638,53711668,Shabby Chic 2 Beds Bungalow in Woodgreen,285590054,About The Stay With Jenny,NaN,Haringey,51.599308,-0.114763,Entire home/apt,104,...,0,NaN,NaN,3,73,0,NaN,Alexandra Palace,390.877638,2.0
66639,53712786,Outstanding Luxury 2Bed 2.5Bath Notting Hill Gate,379423186,Matthew,NaN,Kensington and Chelsea,51.510407,-0.193729,Entire home/apt,402,...,0,NaN,NaN,6,365,0,NaN,Notting Hill Gate,190.640919,3.0


In [4]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license', 'station',
       'station_dist', 'walking_distance'],
      dtype='object')

In [5]:
#drop columns not being compared
cols_to_drop = ['last_review','host_name','neighbourhood_group', 'last_review','license','availability_365','calculated_host_listings_count','number_of_reviews']
df.drop(cols_to_drop, axis = 1, inplace = True)

#fill reviews per month nulls to 0
df['reviews_per_month'].fillna(0, inplace = True)
df

,id,name,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,reviews_per_month,number_of_reviews_ltm,station,station_dist,walking_distance
0,13913,Holiday London DB Room Let-on going,54730,Islington,51.568610,-0.112700,Private room,65,1,0.16,1,Crouch Hill,431.108995,2.0
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Kensington and Chelsea,51.487800,-0.168130,Entire home/apt,75,3,0.61,0,South Kensington,805.406363,0.0
2,17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,67564,Westminster,51.521950,-0.140940,Entire home/apt,265,4,0.33,1,Great Portland Street,292.948350,5.0
3,17506,Boutique Chelsea/Fulham Double bed 5-star ensuite,67915,Hammersmith and Fulham,51.479350,-0.197430,Private room,150,3,0.00,0,Fulham Broadway,165.612825,2.0
4,25123,Clean big Room in London (Room 1),103583,Barnet,51.574380,-0.210810,Private room,29,10,0.91,0,Brent Cross,302.665825,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66636,53709893,Private 3 bedroom Crystal palace in high street,96150452,Croydon,51.419345,-0.079681,Entire home/apt,120,2,0.00,0,Crystal Palace,509.159633,2.0
66637,53710986,"Chiswick Park 3 bed 2 bath, Chiswick, West London",125392965,Ealing,51.502002,-0.276895,Entire home/apt,288,3,0.00,0,Acton Town,264.806903,2.0
66638,53711668,Shabby Chic 2 Beds Bungalow in Woodgreen,285590054,Haringey,51.599308,-0.114763,Entire home/apt,104,5,0.00,0,Alexandra Palace,390.877638,2.0
66639,53712786,Outstanding Luxury 2Bed 2.5Bath Notting Hill Gate,379423186,Kensington and Chelsea,51.510407,-0.193729,Entire home/apt,402,7,0.00,0,Notting Hill Gate,190.640919,3.0


In [6]:
#get count of null values
df.isna().sum()

id                        0
name                     24
host_id                   0
neighbourhood             0
latitude                  0
longitude                 0
room_type                 0
price                     0
minimum_nights            0
reviews_per_month         0
number_of_reviews_ltm     0
station                   0
station_dist              0
walking_distance          0
dtype: int64

In [7]:
#fill name nulls to 'Unnamed'
df['name'].fillna('Unnamed', inplace = True)
df.isna().sum()

id                       0
name                     0
host_id                  0
neighbourhood            0
latitude                 0
longitude                0
room_type                0
price                    0
minimum_nights           0
reviews_per_month        0
number_of_reviews_ltm    0
station                  0
station_dist             0
walking_distance         0
dtype: int64

In [8]:
# since normalisation is going to happen, latitude and longitude need to be converted to cartesian plane
# https://datascience.stackexchange.com/questions/13567/ways-to-deal-with-longitude-latitude-feature

df['x'] = np.cos(df['latitude']) * np.cos(df['longitude'])
df['y'] = np.cos(df['latitude']) * np.sin(df['longitude'])
df['z'] = np.sin(df['latitude'])

df.drop(['latitude', 'longitude'], axis = 1, inplace = True)
df

,id,name,host_id,neighbourhood,room_type,price,minimum_nights,reviews_per_month,number_of_reviews_ltm,station,station_dist,walking_distance,x,y,z
0,13913,Holiday London DB Room Let-on going,54730,Islington,Private room,65,1,0.16,1,Crouch Hill,431.108995,2.0,0.262806,-0.029744,0.964390
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Kensington and Chelsea,Entire home/apt,75,3,0.61,0,South Kensington,805.406363,0.0,0.336654,-0.057141,0.939893
2,17402,Superb 3-Bed/2 Bath & Wifi: Trendy W1,67564,Westminster,Entire home/apt,265,4,0.33,1,Great Portland Street,292.948350,5.0,0.306113,-0.043431,0.951004
3,17506,Boutique Chelsea/Fulham Double bed 5-star ensuite,67915,Hammersmith and Fulham,Private room,150,3,0.00,0,Fulham Broadway,165.612825,2.0,0.342611,-0.068534,0.936974
4,25123,Clean big Room in London (Room 1),103583,Barnet,Private room,29,10,0.91,0,Brent Cross,302.665825,1.0,0.253183,-0.054179,0.965900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66636,53709893,Private 3 bedroom Crystal palace in high street,96150452,Croydon,Entire home/apt,120,2,0.00,0,Crystal Palace,509.159633,2.0,0.403674,-0.032233,0.914335
66637,53710986,"Chiswick Park 3 bed 2 bath, Chiswick, West London",125392965,Ealing,Entire home/apt,288,3,0.00,0,Acton Town,264.806903,2.0,0.315589,-0.089689,0.944648
66638,53711668,Shabby Chic 2 Beds Bungalow in Woodgreen,285590054,Haringey,Entire home/apt,104,5,0.00,0,Alexandra Palace,390.877638,2.0,0.233215,-0.026883,0.972054
66639,53712786,Outstanding Luxury 2Bed 2.5Bath Notting Hill Gate,379423186,Kensington and Chelsea,Entire home/apt,402,7,0.00,0,Notting Hill Gate,190.640919,3.0,0.314146,-0.061632,0.947372


In [9]:
df.dtypes

id                         int64
name                      object
host_id                    int64
neighbourhood             object
room_type                 object
price                      int64
minimum_nights             int64
reviews_per_month        float64
number_of_reviews_ltm      int64
station                   object
station_dist             float64
walking_distance         float64
x                        float64
y                        float64
z                        float64
dtype: object

In [10]:
#get a mapping of ids to index
full_mapping = pd.Series(df.index,index = df['id'])
full_mapping

id
13913           0
15400           1
17402           2
17506           3
25123           4
            ...  
53709893    66636
53710986    66637
53711668    66638
53712786    66639
53713050    66640
Length: 66641, dtype: int64

In [11]:
#drop columns that are relavant for results but are not being compared
df_no_ids = df.drop(['id','host_id','name','price'], axis = 1)

#one hot encode categorical columns
df_no_ids = pd.get_dummies(df_no_ids)

#normalise df for cosine similarity calculation (linear kernel only does dot product so we need to manually normalise)
normalizer = Normalizer()
df_no_ids = pd.DataFrame(normalizer.fit_transform(df_no_ids))

In [12]:
def create_split_dfs(airbnb_id, selected_df):

  #get the index value from the full mapping from df
  airbnb_index = full_mapping[airbnb_id]

  #create a df with only the row containing the airbnb id of choice and a df without it
  selected_row = selected_df.iloc[airbnb_index]
  df_new = selected_df.drop(airbnb_index, axis=0)  

  #split df without row of choice into 4
  df_split = np.array_split(df_new, 4)


  small_df_list = []

  #append the selected row to each df for comparison
  for d in df_split:
    small_df = d.reset_index(drop = True).append(selected_row, ignore_index = True)
    small_df_list.append(small_df)

  return small_df_list

In [13]:
def get_similar_airbnb(airbnb_id, similarity_matrix, selected_df, no_id_df):

  #get mapping for the small df selected and then find the airbnb id selected
  mapping = pd.Series(selected_df.index,index = selected_df['id'])
  airbnb_index = mapping[airbnb_id]

  #get similarity scores for all others in that df to selected airbnb and order descending
  similarity_score = list(enumerate(similarity_matrix[airbnb_index]))
  similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)

  #select top 10 (ignoring first because that will be the same one)
  similarity_score = similarity_score[1:11]

  #get indices of the highest similarity scores here
  airbnb_indices = [i[0] for i in similarity_score]

  #return a df with selected rows from main df and one without ids
  return  selected_df.iloc[airbnb_indices], no_id_df.iloc[airbnb_indices]

In [14]:
def get_top_10(airbnb_id):

  #get index from main df from full mapping
  airbnb_index = full_mapping[airbnb_id]

  #get selected row from main df and one without id
  selected_row = df.iloc[airbnb_index]
  selected_row_no_id = df_no_ids.iloc[airbnb_index]

  #get smaller dfs with and without ids and with normalisation 
  dfs = create_split_dfs(airbnb_id, df)
  dfs_no_ids= create_split_dfs(airbnb_id, df_no_ids)

  top_40_list = []
  top_40_list_no_id = []

  #get top 10 for each small dfs and append to list
  for i in range(len(dfs)):
    similarity_matrix = linear_kernel(dfs_no_ids[i],dfs_no_ids[i])
    dfs_top_10, dfs_top_10_no_id = get_similar_airbnb(airbnb_id, similarity_matrix,dfs[i], dfs_no_ids[i])
    top_40_list.append(dfs_top_10)
    top_40_list_no_id.append(dfs_top_10_no_id)

  #create main df with top 40 from above for final output
  top_40_df = pd.concat(top_40_list, ignore_index = True)
  top_40_df = top_40_df.append(selected_row).reset_index(drop= True)

  #create df with top 40 without ids and with normalisation for comparison
  top_40_df_no_id = pd.concat(top_40_list_no_id, ignore_index = True)
  top_40_df_no_id = top_40_df_no_id.append(selected_row_no_id).reset_index(drop = True)

  #compare top 40 from above to get top 10
  similarity_matrix = linear_kernel(top_40_df_no_id,top_40_df_no_id)
  top_10, top_10_no_id  = get_similar_airbnb(airbnb_id, similarity_matrix, top_40_df, top_40_df_no_id)

  return top_10

In [15]:
#test results

test = get_top_10(15400)

In [16]:
len(test)

10

In [17]:
test

,id,name,host_id,neighbourhood,room_type,price,minimum_nights,reviews_per_month,number_of_reviews_ltm,station,station_dist,walking_distance,x,y,z
10,22385454,Chic Chelsea apt 2 Bed & 2 Bath- Prime Kings r...,61649970,Kensington and Chelsea,Entire home/apt,129,3,0.61,0,South Kensington,899.759738,0.0,0.337833,-0.057880,0.939425
11,26883742,King's Road Friendly Studio,67306414,Kensington and Chelsea,Entire home/apt,125,3,0.20,0,South Kensington,808.190300,0.0,0.337175,-0.058163,0.939644
0,15305414,Kings Road amazing 2BR with great terrace,6926556,Kensington and Chelsea,Entire home/apt,119,4,0.58,0,South Kensington,945.923365,0.0,0.338412,-0.059217,0.939133
1,13984052,Lovely 2Bed Chelsea bet famous Kings Road &Tha...,56846808,Kensington and Chelsea,Entire home/apt,80,4,0.46,0,South Kensington,1077.477847,0.0,0.339474,-0.058539,0.938792
20,35019739,"Beautiful 5BD townhouse by Kings Road, Chelsea",263825749,Kensington and Chelsea,Entire home/apt,900,3,0.00,0,South Kensington,801.536224,0.0,0.336760,-0.057353,0.939842
12,16240561,"Elegant, bright, spacious 2 bedroom Chelsea flat",1672712,Kensington and Chelsea,Entire home/apt,150,3,0.02,0,South Kensington,851.909891,0.0,0.337365,-0.060014,0.939459
21,39162464,Lovely 1 bed in Chelsea on 1st floor,230345473,Kensington and Chelsea,Entire home/apt,160,3,0.04,0,South Kensington,886.725998,0.0,0.337066,-0.060883,0.939510
13,19487983,New Exclusive Two Bedroom Flat Centre Chelsea!,28820321,Kensington and Chelsea,Entire home/apt,160,3,0.39,0,South Kensington,979.002201,0.0,0.338136,-0.060720,0.939136
2,2044274,Beautiful & Spacious Chelsea Flat,7357625,Kensington and Chelsea,Entire home/apt,132,3,0.28,0,South Kensington,980.400462,0.0,0.338495,-0.060097,0.939047
22,38281193,Inviting Chelsea 2BR Apartment with Patio (Veeve),137094377,Kensington and Chelsea,Entire home/apt,281,4,0.00,0,South Kensington,989.445359,0.0,0.338079,-0.060930,0.939143


In [18]:
df[df['id'] == 15400]

,id,name,host_id,neighbourhood,room_type,price,minimum_nights,reviews_per_month,number_of_reviews_ltm,station,station_dist,walking_distance,x,y,z
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Kensington and Chelsea,Entire home/apt,75,3,0.61,0,South Kensington,805.406363,0.0,0.336654,-0.057141,0.939893


## Ignore

In [19]:
#df_without_ids = df.drop(['id','host_id'], axis = 1)
# df_without_ids = df_small.drop(['id','host_id','name','price'], axis = 1)

In [20]:
# tfidf = TfidfVectorizer(stop_words = 'english', min_df = 2, max_features = 100)

In [21]:
# unscaled_features = [ 'minimum_nights','number_of_reviews','reviews_per_month',
#                      'calculated_host_listings_count','station_dist','number_of_reviews_ltm',
#                      'walking_distance','x','y','z']

# all_features = df_without_ids.columns
# numeric_transformer = Normalizer()

# categorical_features = ['neighbourhood', 'room_type', 'station']
# categorical_transformer = OneHotEncoder(handle_unknown='ignore')


# preprocessor = ColumnTransformer(transformers = [("tfidf", tfidf, "name"),
#                                                  ('num', numeric_transformer, unscaled_features),
#                                                  ('cat', categorical_transformer, categorical_features)],
#                                  remainder="passthrough", sparse_threshold=0)


# preprocessor = ColumnTransformer(transformers = [('cat', categorical_transformer, categorical_features),
#                                                  ('all', numeric_transformer, all_features)],
#                                  remainder="passthrough", sparse_threshold=0)

In [22]:
# processed_data = preprocessor.fit_transform(df_without_ids)
# processed_data


In [23]:
# processed_data.shape

In [24]:
# test = pd.DataFrame(processed_data)
# test

In [25]:
# similarity_matrix

In [26]:
# get_similar_airbnb(15400)

In [27]:
#test.T.corr()

In [28]:
#corr_test = test.T.corr()

In [29]:
#corr_test

In [30]:
# np.array_split(df, 4)[2]